In [1]:
import FlowCytometryTools
import glob
import pandas as pd
import numpy as np
import impyute as im
import os
import fancyimpute
np.set_printoptions(suppress=True)
from fancyimpute import KNN, NuclearNormMinimization, SoftImpute, BiScaler
from FlowCytometryTools import FCMeasurement
import pickle
import gc

Using TensorFlow backend.


In [6]:
gc.collect()

20

In [4]:
files_path = '/Users/kangakum/FlowAI/RealTestCases'

In [5]:
read_files = glob.glob(os.path.join(files_path, "*.fcs"))

In [6]:
read_files.sort()

In [7]:
files_per_case = 4
numcases = int(len(read_files) / files_per_case)
cases = []
for x in range(numcases):
    cases.append([])


In [8]:
for x in range(len(read_files)):
    cases[int(x/4)].append(read_files[x])

In [9]:
channels = ['FSC-A', 'FSC-H', 'SSC-A', 'FITC-A', 'PE-A', 'PerCP-Cy5-5-A', 'PE-Cy7-A', 'APC-A', 'APC-R700-A', 'APC-H7-A', 'V450-A', 'V500-C-A', 'BV605-A']
Tube1Antibodies = ['FSC-A', 'FSC-H', 'SSC-A', 'Kap P', 'Lam P', 'CD19', 'CD2', 'CD10', 'CD20', 'CD23', 'CD5', 'CD45', 'CD200']
Tube2Antibodies = ['FSC-A', 'FSC-H', 'SSC-A', 'Kap M', 'Lam M', 'CD38', 'CD11c', 'CD103', 'CD3', 'CD25', 'FMC7', 'CD45', 'CD19']
Tube3Antibodies = ['FSC-A', 'FSC-H', 'SSC-A', 'CD57', 'CD16', 'CD8', 'CD5', 'CD2', 'CD4', 'CD56', 'CD3', 'CD45', 'CD7']
Tube4Antibodies = ['FSC-A', 'FSC-H', 'SSC-A', 'TCR ab', 'TCRgd', 'CD3', 'CD2', 'CD22', 'CD52', 'CD19', 'CD30', 'CD45', 'HLADR']
print(len(channels))

13


In [10]:
Tubes = []
Tubes.append({channels[i]: Tube1Antibodies[i] for i in range(len(channels))})
Tubes.append({channels[i]: Tube2Antibodies[i] for i in range(len(channels))})
Tubes.append({channels[i]: Tube3Antibodies[i] for i in range(len(channels))})
Tubes.append({channels[i]: Tube4Antibodies[i] for i in range(len(channels))})

In [11]:
columns = ['FSC-A', 'FSC-H', 'SSC-A', 'Kap P', 'Lam P', 'CD19', 'CD2', 'CD10', 'CD20', 'CD23', 'CD5', 'CD45', 'CD200', 'Kap M', 'Lam M', 'CD38', 'CD11c', 'CD103', 'CD3', 'CD25', 'FMC7', 'CD57', 'CD16', 'CD8', 'CD4', 'CD56', 'CD7','TCR ab', 'TCRgd','CD22', 'CD52', 'CD30', 'HLADR']
print(columns)

['FSC-A', 'FSC-H', 'SSC-A', 'Kap P', 'Lam P', 'CD19', 'CD2', 'CD10', 'CD20', 'CD23', 'CD5', 'CD45', 'CD200', 'Kap M', 'Lam M', 'CD38', 'CD11c', 'CD103', 'CD3', 'CD25', 'FMC7', 'CD57', 'CD16', 'CD8', 'CD4', 'CD56', 'CD7', 'TCR ab', 'TCRgd', 'CD22', 'CD52', 'CD30', 'HLADR']


In [12]:
caseData = []
print(caseData)
for x in range(len(cases)):
    rows = 0
    for case in cases[x]:
        sample = FCMeasurement(ID = 'Test Sample', datafile = case)
        rows += sample.data.shape[0]
    caseData.append(np.empty((rows,len(columns),)))
    caseData[x][:] = np.nan
print(caseData)

[]
[array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]]), array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]]), array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]]), array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, 

In [13]:
def case_data_processing(caseNum):
    adjustment = 0
    for x in range(len(cases[caseNum])):
        sample = FCMeasurement(ID = 'Test Sample', datafile = cases[caseNum][x])
        data_as_array = sample.data.values
        data_as_array = np.delete(data_as_array, sample.channel_names.index('Time'), 1)
        key = Tubes[x]
        for y in range(len(channels)):
            antibody = key.get(channels[y])
            antibody_num = columns.index(antibody)
            for z in range(sample.data.values.shape[0]):
                caseData[caseNum][z + adjustment, antibody_num] = data_as_array[z, y]
            print("Data Processing: Case " + str(caseNum + 1) + " tube " + str(x + 1) + " channel " + str(y + 1))
        adjustment += sample.data.values.shape[0]  

In [14]:
for x in range(len(cases)):
    case_data_processing(x)

Data Processing: Case 1 tube 1 channel 1
Data Processing: Case 1 tube 1 channel 2
Data Processing: Case 1 tube 1 channel 3
Data Processing: Case 1 tube 1 channel 4
Data Processing: Case 1 tube 1 channel 5
Data Processing: Case 1 tube 1 channel 6
Data Processing: Case 1 tube 1 channel 7
Data Processing: Case 1 tube 1 channel 8
Data Processing: Case 1 tube 1 channel 9
Data Processing: Case 1 tube 1 channel 10
Data Processing: Case 1 tube 1 channel 11
Data Processing: Case 1 tube 1 channel 12
Data Processing: Case 1 tube 1 channel 13
Data Processing: Case 1 tube 2 channel 1
Data Processing: Case 1 tube 2 channel 2
Data Processing: Case 1 tube 2 channel 3
Data Processing: Case 1 tube 2 channel 4
Data Processing: Case 1 tube 2 channel 5
Data Processing: Case 1 tube 2 channel 6
Data Processing: Case 1 tube 2 channel 7
Data Processing: Case 1 tube 2 channel 8
Data Processing: Case 1 tube 2 channel 9
Data Processing: Case 1 tube 2 channel 10
Data Processing: Case 1 tube 2 channel 11
Data Proce

Data Processing: Case 4 tube 4 channel 5
Data Processing: Case 4 tube 4 channel 6
Data Processing: Case 4 tube 4 channel 7
Data Processing: Case 4 tube 4 channel 8
Data Processing: Case 4 tube 4 channel 9
Data Processing: Case 4 tube 4 channel 10
Data Processing: Case 4 tube 4 channel 11
Data Processing: Case 4 tube 4 channel 12
Data Processing: Case 4 tube 4 channel 13
Data Processing: Case 5 tube 1 channel 1
Data Processing: Case 5 tube 1 channel 2
Data Processing: Case 5 tube 1 channel 3
Data Processing: Case 5 tube 1 channel 4
Data Processing: Case 5 tube 1 channel 5
Data Processing: Case 5 tube 1 channel 6
Data Processing: Case 5 tube 1 channel 7
Data Processing: Case 5 tube 1 channel 8
Data Processing: Case 5 tube 1 channel 9
Data Processing: Case 5 tube 1 channel 10
Data Processing: Case 5 tube 1 channel 11
Data Processing: Case 5 tube 1 channel 12
Data Processing: Case 5 tube 1 channel 13
Data Processing: Case 5 tube 2 channel 1
Data Processing: Case 5 tube 2 channel 2
Data Pro

Data Processing: Case 8 tube 3 channel 10
Data Processing: Case 8 tube 3 channel 11
Data Processing: Case 8 tube 3 channel 12
Data Processing: Case 8 tube 3 channel 13
Data Processing: Case 8 tube 4 channel 1
Data Processing: Case 8 tube 4 channel 2
Data Processing: Case 8 tube 4 channel 3
Data Processing: Case 8 tube 4 channel 4
Data Processing: Case 8 tube 4 channel 5
Data Processing: Case 8 tube 4 channel 6
Data Processing: Case 8 tube 4 channel 7
Data Processing: Case 8 tube 4 channel 8
Data Processing: Case 8 tube 4 channel 9
Data Processing: Case 8 tube 4 channel 10
Data Processing: Case 8 tube 4 channel 11
Data Processing: Case 8 tube 4 channel 12
Data Processing: Case 8 tube 4 channel 13
Data Processing: Case 9 tube 1 channel 1
Data Processing: Case 9 tube 1 channel 2
Data Processing: Case 9 tube 1 channel 3
Data Processing: Case 9 tube 1 channel 4
Data Processing: Case 9 tube 1 channel 5
Data Processing: Case 9 tube 1 channel 6
Data Processing: Case 9 tube 1 channel 7
Data Pro

Data Processing: Case 12 tube 2 channel 11
Data Processing: Case 12 tube 2 channel 12
Data Processing: Case 12 tube 2 channel 13
Data Processing: Case 12 tube 3 channel 1
Data Processing: Case 12 tube 3 channel 2
Data Processing: Case 12 tube 3 channel 3
Data Processing: Case 12 tube 3 channel 4
Data Processing: Case 12 tube 3 channel 5
Data Processing: Case 12 tube 3 channel 6
Data Processing: Case 12 tube 3 channel 7
Data Processing: Case 12 tube 3 channel 8
Data Processing: Case 12 tube 3 channel 9
Data Processing: Case 12 tube 3 channel 10
Data Processing: Case 12 tube 3 channel 11
Data Processing: Case 12 tube 3 channel 12
Data Processing: Case 12 tube 3 channel 13
Data Processing: Case 12 tube 4 channel 1
Data Processing: Case 12 tube 4 channel 2
Data Processing: Case 12 tube 4 channel 3
Data Processing: Case 12 tube 4 channel 4
Data Processing: Case 12 tube 4 channel 5
Data Processing: Case 12 tube 4 channel 6
Data Processing: Case 12 tube 4 channel 7
Data Processing: Case 12 tu

Data Processing: Case 16 tube 1 channel 12
Data Processing: Case 16 tube 1 channel 13
Data Processing: Case 16 tube 2 channel 1
Data Processing: Case 16 tube 2 channel 2
Data Processing: Case 16 tube 2 channel 3
Data Processing: Case 16 tube 2 channel 4
Data Processing: Case 16 tube 2 channel 5
Data Processing: Case 16 tube 2 channel 6
Data Processing: Case 16 tube 2 channel 7
Data Processing: Case 16 tube 2 channel 8
Data Processing: Case 16 tube 2 channel 9
Data Processing: Case 16 tube 2 channel 10
Data Processing: Case 16 tube 2 channel 11
Data Processing: Case 16 tube 2 channel 12
Data Processing: Case 16 tube 2 channel 13
Data Processing: Case 16 tube 3 channel 1
Data Processing: Case 16 tube 3 channel 2
Data Processing: Case 16 tube 3 channel 3
Data Processing: Case 16 tube 3 channel 4
Data Processing: Case 16 tube 3 channel 5
Data Processing: Case 16 tube 3 channel 6
Data Processing: Case 16 tube 3 channel 7
Data Processing: Case 16 tube 3 channel 8
Data Processing: Case 16 tub

...

Data Processing: Case 98 tube 3 channel 8
Data Processing: Case 98 tube 3 channel 9
Data Processing: Case 98 tube 3 channel 10
Data Processing: Case 98 tube 3 channel 11
Data Processing: Case 98 tube 3 channel 12
Data Processing: Case 98 tube 3 channel 13
Data Processing: Case 98 tube 4 channel 1
Data Processing: Case 98 tube 4 channel 2
Data Processing: Case 98 tube 4 channel 3
Data Processing: Case 98 tube 4 channel 4
Data Processing: Case 98 tube 4 channel 5
Data Processing: Case 98 tube 4 channel 6
Data Processing: Case 98 tube 4 channel 7
Data Processing: Case 98 tube 4 channel 8
Data Processing: Case 98 tube 4 channel 9
Data Processing: Case 98 tube 4 channel 10
Data Processing: Case 98 tube 4 channel 11
Data Processing: Case 98 tube 4 channel 12
Data Processing: Case 98 tube 4 channel 13
Data Processing: Case 99 tube 1 channel 1
Data Processing: Case 99 tube 1 channel 2
Data Processing: Case 99 tube 1 channel 3
Data Processing: Case 99 tube 1 channel 4
Data Processing: Case 99 t

Data Processing: Case 102 tube 2 channel 7
Data Processing: Case 102 tube 2 channel 8
Data Processing: Case 102 tube 2 channel 9
Data Processing: Case 102 tube 2 channel 10
Data Processing: Case 102 tube 2 channel 11
Data Processing: Case 102 tube 2 channel 12
Data Processing: Case 102 tube 2 channel 13
Data Processing: Case 102 tube 3 channel 1
Data Processing: Case 102 tube 3 channel 2
Data Processing: Case 102 tube 3 channel 3
Data Processing: Case 102 tube 3 channel 4
Data Processing: Case 102 tube 3 channel 5
Data Processing: Case 102 tube 3 channel 6
Data Processing: Case 102 tube 3 channel 7
Data Processing: Case 102 tube 3 channel 8
Data Processing: Case 102 tube 3 channel 9
Data Processing: Case 102 tube 3 channel 10
Data Processing: Case 102 tube 3 channel 11
Data Processing: Case 102 tube 3 channel 12
Data Processing: Case 102 tube 3 channel 13
Data Processing: Case 102 tube 4 channel 1
Data Processing: Case 102 tube 4 channel 2
Data Processing: Case 102 tube 4 channel 3
Dat

Data Processing: Case 106 tube 1 channel 4
Data Processing: Case 106 tube 1 channel 5
Data Processing: Case 106 tube 1 channel 6
Data Processing: Case 106 tube 1 channel 7
Data Processing: Case 106 tube 1 channel 8
Data Processing: Case 106 tube 1 channel 9
Data Processing: Case 106 tube 1 channel 10
Data Processing: Case 106 tube 1 channel 11
Data Processing: Case 106 tube 1 channel 12
Data Processing: Case 106 tube 1 channel 13
Data Processing: Case 106 tube 2 channel 1
Data Processing: Case 106 tube 2 channel 2
Data Processing: Case 106 tube 2 channel 3
Data Processing: Case 106 tube 2 channel 4
Data Processing: Case 106 tube 2 channel 5
Data Processing: Case 106 tube 2 channel 6
Data Processing: Case 106 tube 2 channel 7
Data Processing: Case 106 tube 2 channel 8
Data Processing: Case 106 tube 2 channel 9
Data Processing: Case 106 tube 2 channel 10
Data Processing: Case 106 tube 2 channel 11
Data Processing: Case 106 tube 2 channel 12
Data Processing: Case 106 tube 2 channel 13
Dat

Data Processing: Case 109 tube 3 channel 12
Data Processing: Case 109 tube 3 channel 13
Data Processing: Case 109 tube 4 channel 1
Data Processing: Case 109 tube 4 channel 2
Data Processing: Case 109 tube 4 channel 3
Data Processing: Case 109 tube 4 channel 4
Data Processing: Case 109 tube 4 channel 5
Data Processing: Case 109 tube 4 channel 6
Data Processing: Case 109 tube 4 channel 7
Data Processing: Case 109 tube 4 channel 8
Data Processing: Case 109 tube 4 channel 9
Data Processing: Case 109 tube 4 channel 10
Data Processing: Case 109 tube 4 channel 11
Data Processing: Case 109 tube 4 channel 12
Data Processing: Case 109 tube 4 channel 13
Data Processing: Case 110 tube 1 channel 1
Data Processing: Case 110 tube 1 channel 2
Data Processing: Case 110 tube 1 channel 3
Data Processing: Case 110 tube 1 channel 4
Data Processing: Case 110 tube 1 channel 5
Data Processing: Case 110 tube 1 channel 6
Data Processing: Case 110 tube 1 channel 7
Data Processing: Case 110 tube 1 channel 8
Data 

Data Processing: Case 113 tube 2 channel 8
Data Processing: Case 113 tube 2 channel 9
Data Processing: Case 113 tube 2 channel 10
Data Processing: Case 113 tube 2 channel 11
Data Processing: Case 113 tube 2 channel 12
Data Processing: Case 113 tube 2 channel 13
Data Processing: Case 113 tube 3 channel 1
Data Processing: Case 113 tube 3 channel 2
Data Processing: Case 113 tube 3 channel 3
Data Processing: Case 113 tube 3 channel 4
Data Processing: Case 113 tube 3 channel 5
Data Processing: Case 113 tube 3 channel 6
Data Processing: Case 113 tube 3 channel 7
Data Processing: Case 113 tube 3 channel 8
Data Processing: Case 113 tube 3 channel 9
Data Processing: Case 113 tube 3 channel 10
Data Processing: Case 113 tube 3 channel 11
Data Processing: Case 113 tube 3 channel 12
Data Processing: Case 113 tube 3 channel 13
Data Processing: Case 113 tube 4 channel 1
Data Processing: Case 113 tube 4 channel 2
Data Processing: Case 113 tube 4 channel 3
Data Processing: Case 113 tube 4 channel 4
Dat

In [108]:
print(caseData[100][500000])

[ 70519.9765625   59696.          17170.                     nan
             nan  13223.28027344    520.20001221             nan
             nan             nan             nan   3366.91992188
             nan             nan             nan             nan
             nan             nan   1541.90002441             nan
             nan             nan             nan             nan
             nan             nan             nan    255.
    261.80001831  14951.83984375   7135.91992188   3032.35986328
 134452.125     ]


In [109]:
ImputedCases = []
for x in range(len(caseData)):
    ImputedCases.append(SoftImpute().fit_transform(caseData[x]))
    print("Imputed data into case " + str(x))
    if x % 10 == 0 || x == 115:
        file = open('/Users/kangakum/FlowAI/RealTestCases/ImputedCases/' + str(x/10), 'wb')
        pickle.dump(ImputedCases, file)
        ImputedCases = []
        print("dumped into" + str(x))

[SoftImpute] Max Singular Value of X_init = 48550067.429134
[SoftImpute] Iter 1: observed MAE=1453.112710 rank=15
[SoftImpute] Iter 2: observed MAE=1447.463326 rank=14
[SoftImpute] Iter 3: observed MAE=1449.436802 rank=14
[SoftImpute] Iter 4: observed MAE=1454.753895 rank=14
[SoftImpute] Iter 5: observed MAE=1461.101865 rank=14
[SoftImpute] Iter 6: observed MAE=1467.504541 rank=14
[SoftImpute] Iter 7: observed MAE=1473.573804 rank=14
[SoftImpute] Iter 8: observed MAE=1479.141099 rank=14
[SoftImpute] Iter 9: observed MAE=1483.263198 rank=13
[SoftImpute] Iter 10: observed MAE=1487.054168 rank=13
[SoftImpute] Iter 11: observed MAE=1490.646734 rank=13
[SoftImpute] Iter 12: observed MAE=1493.942538 rank=13
[SoftImpute] Iter 13: observed MAE=1496.927089 rank=13
[SoftImpute] Iter 14: observed MAE=1499.611805 rank=13
[SoftImpute] Iter 15: observed MAE=1502.017653 rank=13
[SoftImpute] Iter 16: observed MAE=1504.170619 rank=13
[SoftImpute] Iter 17: observed MAE=1506.094072 rank=13
[SoftImpute] I

[SoftImpute] Iter 32: observed MAE=1731.208028 rank=14
[SoftImpute] Iter 33: observed MAE=1731.718243 rank=14
[SoftImpute] Iter 34: observed MAE=1732.179808 rank=14
[SoftImpute] Iter 35: observed MAE=1732.597202 rank=14
[SoftImpute] Iter 36: observed MAE=1732.974772 rank=14
[SoftImpute] Iter 37: observed MAE=1733.316505 rank=14
[SoftImpute] Iter 38: observed MAE=1733.625670 rank=14
[SoftImpute] Iter 39: observed MAE=1733.905240 rank=14
[SoftImpute] Iter 40: observed MAE=1734.158035 rank=14
[SoftImpute] Iter 41: observed MAE=1734.386605 rank=14
[SoftImpute] Iter 42: observed MAE=1734.593293 rank=14
[SoftImpute] Iter 43: observed MAE=1734.780133 rank=14
[SoftImpute] Iter 44: observed MAE=1734.948997 rank=14
[SoftImpute] Iter 45: observed MAE=1735.101606 rank=14
[SoftImpute] Iter 46: observed MAE=1735.239494 rank=14
[SoftImpute] Iter 47: observed MAE=1735.364187 rank=14
[SoftImpute] Iter 48: observed MAE=1735.476892 rank=14
[SoftImpute] Iter 49: observed MAE=1735.578755 rank=14
[SoftImput

[SoftImpute] Iter 10: observed MAE=1129.195083 rank=9
[SoftImpute] Iter 11: observed MAE=1131.057871 rank=9
[SoftImpute] Iter 12: observed MAE=1132.687745 rank=9
[SoftImpute] Iter 13: observed MAE=1134.100480 rank=9
[SoftImpute] Iter 14: observed MAE=1135.315179 rank=9
[SoftImpute] Iter 15: observed MAE=1136.357004 rank=9
[SoftImpute] Iter 16: observed MAE=1137.246797 rank=9
[SoftImpute] Iter 17: observed MAE=1138.006188 rank=9
[SoftImpute] Iter 18: observed MAE=1138.653641 rank=9
[SoftImpute] Iter 19: observed MAE=1139.202859 rank=9
[SoftImpute] Iter 20: observed MAE=1139.666483 rank=9
[SoftImpute] Iter 21: observed MAE=1140.056552 rank=9
[SoftImpute] Iter 22: observed MAE=1140.384307 rank=9
[SoftImpute] Iter 23: observed MAE=1140.657623 rank=9
[SoftImpute] Iter 24: observed MAE=1140.883666 rank=9
[SoftImpute] Iter 25: observed MAE=1141.068770 rank=9
[SoftImpute] Iter 26: observed MAE=1141.217865 rank=9
[SoftImpute] Iter 27: observed MAE=1141.335698 rank=9
[SoftImpute] Iter 28: observ

Imputed data into case 5
[SoftImpute] Max Singular Value of X_init = 121917757.745651
[SoftImpute] Iter 1: observed MAE=1853.924394 rank=8
[SoftImpute] Iter 2: observed MAE=1837.139416 rank=7
[SoftImpute] Iter 3: observed MAE=1829.140184 rank=7
[SoftImpute] Iter 4: observed MAE=1828.993035 rank=6
[SoftImpute] Iter 5: observed MAE=1831.645157 rank=6
[SoftImpute] Iter 6: observed MAE=1836.573525 rank=6
[SoftImpute] Iter 7: observed MAE=1842.317341 rank=6
[SoftImpute] Iter 8: observed MAE=1848.038406 rank=6
[SoftImpute] Iter 9: observed MAE=1853.332367 rank=6
[SoftImpute] Iter 10: observed MAE=1858.028388 rank=6
[SoftImpute] Iter 11: observed MAE=1862.091128 rank=6
[SoftImpute] Iter 12: observed MAE=1865.315299 rank=7
[SoftImpute] Iter 13: observed MAE=1867.524778 rank=7
[SoftImpute] Iter 14: observed MAE=1869.237087 rank=7
[SoftImpute] Iter 15: observed MAE=1870.576914 rank=7
[SoftImpute] Iter 16: observed MAE=1871.618159 rank=7
[SoftImpute] Iter 17: observed MAE=1872.418304 rank=7
[Soft

[SoftImpute] Iter 54: observed MAE=1760.751371 rank=8
[SoftImpute] Iter 55: observed MAE=1760.739649 rank=8
[SoftImpute] Iter 56: observed MAE=1760.727764 rank=8
[SoftImpute] Iter 57: observed MAE=1760.715809 rank=8
[SoftImpute] Iter 58: observed MAE=1760.703843 rank=8
[SoftImpute] Iter 59: observed MAE=1760.691909 rank=8
[SoftImpute] Iter 60: observed MAE=1760.680066 rank=8
[SoftImpute] Iter 61: observed MAE=1760.668339 rank=8
[SoftImpute] Iter 62: observed MAE=1760.656804 rank=8
[SoftImpute] Iter 63: observed MAE=1760.645500 rank=8
[SoftImpute] Iter 64: observed MAE=1760.634549 rank=8
[SoftImpute] Iter 65: observed MAE=1760.623898 rank=8
[SoftImpute] Iter 66: observed MAE=1760.613515 rank=8
[SoftImpute] Iter 67: observed MAE=1760.603387 rank=8
[SoftImpute] Iter 68: observed MAE=1760.593542 rank=8
[SoftImpute] Iter 69: observed MAE=1760.584033 rank=8
[SoftImpute] Iter 70: observed MAE=1760.495880 rank=7
[SoftImpute] Iter 71: observed MAE=1760.425745 rank=7
[SoftImpute] Iter 72: observ

...

[SoftImpute] Iter 11: observed MAE=1917.188130 rank=8
[SoftImpute] Iter 12: observed MAE=1920.409049 rank=8
[SoftImpute] Iter 13: observed MAE=1923.339833 rank=8
[SoftImpute] Iter 14: observed MAE=1925.959071 rank=8
[SoftImpute] Iter 15: observed MAE=1928.272629 rank=8
[SoftImpute] Iter 16: observed MAE=1929.890775 rank=7
[SoftImpute] Iter 17: observed MAE=1931.173553 rank=7
[SoftImpute] Iter 18: observed MAE=1932.319928 rank=7
[SoftImpute] Iter 19: observed MAE=1933.338789 rank=7
[SoftImpute] Iter 20: observed MAE=1934.239455 rank=7
[SoftImpute] Iter 21: observed MAE=1935.033010 rank=7
[SoftImpute] Iter 22: observed MAE=1935.731635 rank=7
[SoftImpute] Iter 23: observed MAE=1936.346497 rank=7
[SoftImpute] Iter 24: observed MAE=1936.887788 rank=7
[SoftImpute] Iter 25: observed MAE=1937.364681 rank=7
[SoftImpute] Iter 26: observed MAE=1937.785769 rank=7
[SoftImpute] Iter 27: observed MAE=1938.158297 rank=7
[SoftImpute] Iter 28: observed MAE=1938.488678 rank=7
[SoftImpute] Iter 29: observ

[SoftImpute] Iter 81: observed MAE=1221.037103 rank=9
[SoftImpute] Iter 82: observed MAE=1220.958604 rank=8
[SoftImpute] Iter 83: observed MAE=1220.831287 rank=8
[SoftImpute] Iter 84: observed MAE=1220.775100 rank=8
[SoftImpute] Iter 85: observed MAE=1220.753782 rank=8
[SoftImpute] Iter 86: observed MAE=1220.749541 rank=8
[SoftImpute] Iter 87: observed MAE=1220.753658 rank=8
[SoftImpute] Iter 88: observed MAE=1220.761831 rank=8
[SoftImpute] Iter 89: observed MAE=1220.771933 rank=8
[SoftImpute] Iter 90: observed MAE=1220.782902 rank=8
[SoftImpute] Iter 91: observed MAE=1220.794207 rank=8
[SoftImpute] Iter 92: observed MAE=1220.805566 rank=8
[SoftImpute] Iter 93: observed MAE=1220.816846 rank=8
[SoftImpute] Iter 94: observed MAE=1220.827976 rank=8
[SoftImpute] Iter 95: observed MAE=1220.838916 rank=8
[SoftImpute] Iter 96: observed MAE=1220.849653 rank=8
[SoftImpute] Iter 97: observed MAE=1220.860176 rank=8
[SoftImpute] Iter 98: observed MAE=1220.870482 rank=8
[SoftImpute] Iter 99: observ

[SoftImpute] Iter 80: observed MAE=1756.165540 rank=8
[SoftImpute] Iter 81: observed MAE=1756.181688 rank=8
[SoftImpute] Iter 82: observed MAE=1756.197234 rank=8
[SoftImpute] Iter 83: observed MAE=1756.212211 rank=8
[SoftImpute] Iter 84: observed MAE=1756.226653 rank=8
[SoftImpute] Iter 85: observed MAE=1756.240590 rank=8
[SoftImpute] Iter 86: observed MAE=1756.254049 rank=8
[SoftImpute] Iter 87: observed MAE=1756.267055 rank=8
[SoftImpute] Iter 88: observed MAE=1756.279635 rank=8
[SoftImpute] Iter 89: observed MAE=1756.291810 rank=8
[SoftImpute] Stopped after iteration 89 for lambda=2973025.935326
Imputed data into case 109
[SoftImpute] Max Singular Value of X_init = 334608520.655037
[SoftImpute] Iter 1: observed MAE=1785.446616 rank=5
[SoftImpute] Iter 2: observed MAE=1746.496005 rank=5
[SoftImpute] Iter 3: observed MAE=1724.825172 rank=4
[SoftImpute] Iter 4: observed MAE=1713.577462 rank=4
[SoftImpute] Iter 5: observed MAE=1709.231618 rank=4
[SoftImpute] Iter 6: observed MAE=1708.35

[SoftImpute] Iter 17: observed MAE=1667.565095 rank=4
[SoftImpute] Iter 18: observed MAE=1667.546695 rank=4
[SoftImpute] Iter 19: observed MAE=1667.524363 rank=4
[SoftImpute] Iter 20: observed MAE=1667.499921 rank=4
[SoftImpute] Iter 21: observed MAE=1667.474855 rank=4
[SoftImpute] Iter 22: observed MAE=1667.450335 rank=4
[SoftImpute] Iter 23: observed MAE=1667.427142 rank=4
[SoftImpute] Iter 24: observed MAE=1667.405739 rank=4
[SoftImpute] Iter 25: observed MAE=1667.386317 rank=4
[SoftImpute] Iter 26: observed MAE=1667.368957 rank=4
[SoftImpute] Iter 27: observed MAE=1667.353654 rank=4
[SoftImpute] Iter 28: observed MAE=1667.340289 rank=4
[SoftImpute] Iter 29: observed MAE=1667.328711 rank=4
[SoftImpute] Stopped after iteration 29 for lambda=6670347.019682
Imputed data into case 112
[SoftImpute] Max Singular Value of X_init = 161577260.723593
[SoftImpute] Iter 1: observed MAE=1552.117339 rank=9
[SoftImpute] Iter 2: observed MAE=1532.182222 rank=9
[SoftImpute] Iter 3: observed MAE=1520

[SoftImpute] Iter 41: observed MAE=1849.193361 rank=8
[SoftImpute] Iter 42: observed MAE=1849.248077 rank=8
[SoftImpute] Iter 43: observed MAE=1849.298755 rank=8
[SoftImpute] Iter 44: observed MAE=1849.345794 rank=8
[SoftImpute] Iter 45: observed MAE=1849.389560 rank=8
[SoftImpute] Iter 46: observed MAE=1849.430347 rank=8
[SoftImpute] Iter 47: observed MAE=1849.468361 rank=8
[SoftImpute] Iter 48: observed MAE=1849.503840 rank=8
[SoftImpute] Iter 49: observed MAE=1849.536990 rank=8
[SoftImpute] Iter 50: observed MAE=1849.568038 rank=8
[SoftImpute] Iter 51: observed MAE=1849.597144 rank=8
[SoftImpute] Iter 52: observed MAE=1849.624419 rank=8
[SoftImpute] Iter 53: observed MAE=1849.649993 rank=8
[SoftImpute] Iter 54: observed MAE=1849.674044 rank=8
[SoftImpute] Iter 55: observed MAE=1849.696641 rank=8
[SoftImpute] Iter 56: observed MAE=1849.717894 rank=8
[SoftImpute] Iter 57: observed MAE=1849.737894 rank=8
[SoftImpute] Iter 58: observed MAE=1849.756724 rank=8
[SoftImpute] Iter 59: observ

In [2]:
print(ImputedCases[100][200000])

NameError: name 'ImputedCases' is not defined

In [15]:
import pickle

In [1]:
pickle.dump(ImputedCases, file)

NameError: name 'pickle' is not defined

In [ ]:
file.close()